# Import Modules

In [1]:
import numpy as np
import matplotlib.gridspec as gridspec


In [2]:
mdir_PR4 = "/global/project/projectdirs/cmb/data/planck2020/pla/frequency_maps/Single-frequency/"
mdir_PR3 = "/global/project/projectdirs/cmb/data/planck2018/pr3/frequencymaps/"
PR4      = "HFI_SkyMap_353_2048_R4.00_full.fits"
PR3      = "HFI_SkyMap_353_2048_R3.01_full.fits"

In [3]:
import healpy as hp
import pysm3.units as u
comp = "IQU"
m = hp.read_map(mdir_PR4 + PR4, [c + "_STOKES" for c in comp], 
                   dtype = np.float64)
m <<= u.K_CMB
m = m.to('uK_RJ', equivalencies = u.cmb_equivalencies(353 * u.GHz))

In [4]:
import pandas as pd
from astropy.table import Table
target_url = 'https://irsa.ipac.caltech.edu/data/Planck/release_2/catalogs/HFI_PCCS_GCC_R2.02.fits'
PGCC = Table.read(target_url, format = 'fits')
df   = PGCC.to_pandas()
df   = df[df.FLUX_QUALITY  == 1]    # Cut to best quality sources
df   = df[df.FLUX_BLENDING == 0]   # Cut out overlapping sources
df   = df.reset_index(drop = True) # Reset dataframe indices
df = df[:100]

In [5]:
output_dir = "/global/cscratch1/sd/justinc/"

def stack_I(l, b, p):
    return(hp.gnomview(I, rot = [l,b,p],
                      reso = 1, xsize = 50,
                      coord = 'G', no_plot = True,
                      return_projected_map = True))
def stack_P(l, b, p):
    return(hp.gnomview(P, rot = [l,b,p],
                      reso = 1, xsize = 50,
                      coord = 'G', no_plot = True,
                      return_projected_map = True))

def repeatI(l,b):
    p = np.random.randint(0,360,5)
    res = [stack_I(l,b,p[i]) for i in range(len(p))]
    return np.mean(res, axis = 0)

def repeatP(l,b):
    p = np.random.randint(0,360,5)
    res = [stack_P(l,b,p[i]) for i in range(len(p))]
    return np.mean(res, axis = 0)

In [6]:
m = m ** 2
I,Q,U = m
P = Q + U

In [8]:
from time import time
import multiprocessing

print("\nStacking unfiltered\n")
t0 = time()
glo = df['GLON']
gla = df['GLAT']
num_proc = 16
pool = multiprocessing.Pool(processes = num_proc)
process_I = [pool.apply_async(repeatI, args = (i,j)) for i,j in zip(glo,gla)]
process_P = [pool.apply_async(repeatP, args = (i,j)) for i,j in zip(glo,gla)]
res_I = [p.get() for p in process_I]
res_P = [p.get() for p in process_P]
pool.close()
pool.join()
print(f"Stacking complete in {(time()-t0)/60}m")
stack = [res_I, res_P]



Stacking unfiltered

Stacking complete in 0.2047996719678243m


# Analysis Process
## Full sample results
1. Import PGCC catalogue as a dataframe
2. Load frequency maps
3. Convert maps to uK_RJ from K_CMB
4. Square IQU and convert Q and U to P
5. Create patches of 50 x 50 arcmin centered around the location of each Galactic cold clump
6. Save each patch for I and P in a list
7. Add the patches together and divide by the total number of patches to obtain the averaged signal
8. Perform background and noise subtraction
    - Measure the mean signal in an annulus with chosen inner and outer radius
    - Subtract this from the entire signal
9. Measure the polarization fraction as the peak signal within a circle of radius 2 arcmins from the patch center